# VacationPy


* Heat map to display humidity.

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * Max and min temp.

  * Wind speed.

  * No clouds.

* Google places to find hotels within 5000

* Plot hotels on top of heat map

# Import dependencies and API keys

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
%matplotlib notebook
%matplotlib inline

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## Store WeatherPy in DF

* csv from WeatherPy

In [14]:
CityWeather_df = pd.read_csv('output_data/cities.csv')
CityWeather_df.head()

,City,cloudiness,Country,Date,Humidity,Lat,Lng,temp,wind_speed
0,Abilene,1,US,1591645648,38,32.45,-99.73,99.00,3.36
1,Abu Dhabi,0,AE,1591645874,70,24.47,54.37,89.60,4.70
2,Abu Kamal,5,SY,1591645951,11,34.45,40.92,91.74,6.55
3,Abu Samrah,0,SY,1591645911,29,35.30,37.18,79.83,8.14
4,Acajutla,20,SV,1591645917,70,13.59,-89.83,89.60,7.18


# Humidity Heatmap

* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Store latitude and longitude in locations
lat_lon = CityWeather_df[['Lat', 'Lng']]

# Fill NaN values and convert to float
humid = CityWeather_df["Humidity"]
temp = CityWeather_df["temp"]
wind_speed = CityWeather_df["wind_speed"]
cloudiness = CityWeather_df["cloudiness"]
lat_lon.head()

,Lat,Lng
0,32.45,-99.73
1,24.47,54.37
2,34.45,40.92
3,35.30,37.18
4,13.59,-89.83


In [17]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
#heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 #dissipating=False, max_intensity=100,
                                 #point_radius=1.5)

# Add heat layer
#fig.add_layer(heat_layer)

# Display figure
#fig


# Create a poverty Heatmap layer
#fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

## New DF to include weather

* reduce city count to weather.

In [18]:
# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 70.
crit_temperature = (CityWeather_df.temp < 80) & (CityWeather_df.temp > 70)
crit_windSpeed = CityWeather_df.wind_speed < 15
crit_cloudiness = CityWeather_df.cloudiness == 0
final_criteria = crit_temperature & crit_windSpeed & crit_cloudiness 

# Use boolean indexing to filter the weather_df dataframe
ideal_weather_df = CityWeather_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)

,index,City,cloudiness,Country,Date,Humidity,Lat,Lng,temp,wind_speed
0,3,Abu Samrah,0,SY,1591645911,29,35.30,37.18,79.83,8.14
1,21,Antequera,0,ES,1591645930,31,37.02,-4.56,78.80,6.93
2,24,Areado,0,BR,1591645954,73,-21.36,-46.15,71.02,1.50
3,48,Belmonte,0,BR,1591645964,73,-15.86,-38.88,75.20,3.36
4,70,Bukama,0,CD,1591645914,43,-9.20,25.85,70.70,0.87
5,80,Caravelas,0,BR,1591645939,74,-17.71,-39.25,72.41,3.76
6,106,Conchas,0,BR,1591645908,74,-23.02,-48.01,73.49,2.17
7,205,Kalemie,0,CD,1591645954,68,-5.95,29.19,71.17,2.10
8,222,Kiryat Gat,0,IL,1591645896,73,31.61,34.76,75.00,4.97
9,295,Miandrivazo,0,MG,1591645923,63,-19.52,45.47,70.68,0.60


## Hotel Map

* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.

In [19]:
hotel_df = ideal_weather_df
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,City,cloudiness,Country,Date,Humidity,Lat,Lng,temp,wind_speed,Hotel Name
0,3,Abu Samrah,0,SY,1591645911,29,35.30,37.18,79.83,8.14,
1,21,Antequera,0,ES,1591645930,31,37.02,-4.56,78.80,6.93,
2,24,Areado,0,BR,1591645954,73,-21.36,-46.15,71.02,1.50,
3,48,Belmonte,0,BR,1591645964,73,-15.86,-38.88,75.20,3.36,
4,70,Bukama,0,CD,1591645914,43,-9.20,25.85,70.70,0.87,


* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()

    # Try except for missing cities
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df.info()

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       16 non-null     int64  
 1   City        16 non-null     object 
 2   cloudiness  16 non-null     int64  
 3   Country     16 non-null     object 
 4   Date        16 non-null     int64  
 5   Humidity    16 non-null     int64  
 6   Lat         16 non-n

In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# Add marker layer ontop of heat map

# Create a map using state centroid coordinates to set markers
#marker_locations = census_data[['Latitude', 'Longitude']]

# Create a marker_layer using the poverty list to fill the info box
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

fig.add_layer(heat_layer)

fig


#fig = gmaps.figure()
markers = gmaps.marker_layer(locations)
    #info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
fig.add_layer(markers)
                             
fig





#fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

#heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 #dissipating=False, max_intensity=100,
                                 #point_radius=1.5)


#figure_layout = {
    #'width': '900px',
    #'height': '600px',
    #'border': '1px solid black',
    #'padding': '1px',
    #'margin': '0 auto 0 auto'
#}
#fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
#hotel_layer = gmaps.marker_layer(
    #lat_lon,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#)


# Add layer
#fig.add_layer(heat_layer)
# Display figure
#fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…